In [12]:
import numpy as np
from collections import Counter

In [6]:
# Contain the list of characters indices in the data
input_data = []
# Index of <space> set to 0
char_to_ind = {'<space>':0}
count = 1
with open('../data/train_chars.txt', 'r') as f:
    # Loop to index the char and store them inside the input
    for line in f:
        for c in line[:-1].split(' '):
            # Input data
            if c in char_to_ind:
                input_data.append(char_to_ind[c])
            else:
                char_to_ind[c] = count
                count += 1
                input_data.append(char_to_ind[c])


In [7]:
# Build the input matrix: (num_records, n-1)
# and output (num_records,1) which stores the output for the given (n-1)gram
input_matrix = np.zeros((len(input_data)-n, n-1))
output_matrix = np.zeros(len(input_data)-n)
for i in xrange(len(input_data)-n):
    # Countext is a (n-1)gram
    w = input_data[i:i+(n-1)]
    input_matrix[i,:] = w
    output_matrix[i] = (1 if input_data[i+n]==0 else 2)

NameError: name 'n' is not defined

In [9]:
def get_input(filename,n,char_to_ind=None):
    # Contain the list of characters indices in the data
    # initialized with a padding
    if n > 2:
        input_data = [1]*(n-2)
    else:
        input_data = []
    if char_to_ind is None:
        # Map each character to an index with
        # Index of <space> set to 0
        char_to_ind = {'<space>':0, '</s>':1}
        count = 2
    with open(filename, 'r') as f:
        # Loop to index the char and store them inside the input
        for line in f:
            for c in line[:-1].split(' '):
                # Input data
                if c in char_to_ind:
                    input_data.append(char_to_ind[c])
                else:
                    char_to_ind[c] = count
                    count += 1
                    input_data.append(char_to_ind[c])
    return input_data, char_to_ind

In [91]:
input_data, char_to_ind = get_input('../data/train_chars.txt',n=4)

In [10]:
def get_input(filename, n, char_to_ind=None):
    # Contain the list of characters indices in the data
    # initialized with a padding
    if n > 2:
        input_data = [2]*(n-2)
    else:
        input_data = []
    if char_to_ind is None:
        # Map each character to an index with
        # Index of <space> set to 1
        char_to_ind = {'<space>': 1, '</s>': 2}
        count = 3
    with open(filename, 'r') as f:
        # Loop to index the char and store them inside the input
        for line in f:
            for c in line[:-1].split(' '):
                # Input data
                if c in char_to_ind:
                    input_data.append(char_to_ind[c])
                else:
                    char_to_ind[c] = count
                    count += 1
                    input_data.append(char_to_ind[c])
    return input_data, char_to_ind


def build_train_data(input_data, n):
    # Build the input matrix: (num_records, n-1)
    # and the output vector (num_records,1)
    # which stores the output for the given (n-1)gram
    input_matrix = np.zeros((len(input_data)-n, n-1))
    output_matrix = np.zeros(len(input_data)-n)
    for i in xrange(len(input_data)-n):
        # Countext is a (n-1)gram
        w = input_data[i:i+(n-1)]
        input_matrix[i, :] = w
        output_matrix[i] = (1 if input_data[i+(n-1)] == 1 else 2)
    return input_matrix, output_matrix


def build_count_matrix(input_matrix, output_matrix, n):
    count_matrix_raw = np.concatenate((input_matrix,
                                       output_matrix.reshape(output_matrix.shape[0], 1)), axis=1)

    num_rows = len(set([tuple(s) for s in input_matrix]))
    count = Counter([tuple(s) for s in count_matrix_raw])

    # count matrix: (num_(n-1grams, 2))
    F = np.zeros((num_rows, n + 1))
    gram_to_ind = {}
    i = 0
    for k, v in count.iteritems():
        gram = k[:(n-1)]
        if gram not in gram_to_ind:
            gram_to_ind[gram] = i
            i += 1
        F[gram_to_ind[gram], n-1 + int(k[-1]) - 1] = v
        F[gram_to_ind[gram], :n-1] = list(gram)

    return F

In [13]:
train = '../data/train_chars.txt'
valid = '../data/valid_chars.txt'
N = 2

input_data_train, char_to_ind = get_input(train, N)
input_data_valid, char_to_ind = get_input(valid, N, char_to_ind)
input_matrix_train, output_matrix_train = build_train_data(
    input_data_train, N)
F_train = build_count_matrix(input_matrix_train, output_matrix_train, N)


In [14]:
char_to_ind

{'#': 47,
 '$': 40,
 '&': 39,
 "'": 34,
 '-': 24,
 '.': 33,
 '/': 49,
 '0': 38,
 '1': 35,
 '2': 42,
 '3': 41,
 '4': 43,
 '5': 37,
 '6': 45,
 '7': 46,
 '8': 44,
 '9': 36,
 '<': 28,
 '</s>': 2,
 '<space>': 1,
 '>': 29,
 'N': 30,
 '\\': 48,
 'a': 3,
 'b': 6,
 'c': 14,
 'd': 23,
 'e': 4,
 'f': 19,
 'g': 21,
 'h': 22,
 'i': 12,
 'j': 31,
 'k': 8,
 'l': 11,
 'm': 20,
 'n': 7,
 'o': 9,
 'p': 26,
 'q': 25,
 'r': 5,
 's': 18,
 't': 10,
 'u': 17,
 'v': 32,
 'w': 15,
 'x': 27,
 'y': 16,
 'z': 13}

In [153]:
F_train

array([[ 18.,  18.,  16.,   1.,   0.,   6.],
       [  4.,   6.,   4.,   7.,   0.,  11.],
       [  4.,   4.,  10.,  18.,   9.,   3.],
       ..., 
       [  9.,  10.,   1.,  17.,   0.,   2.],
       [ 14.,   1.,  19.,   3.,   0.,   2.],
       [ 25.,  17.,   3.,  10.,   0.,   6.]])

In [184]:
char_to_ind

{'#': 47,
 '$': 40,
 '&': 39,
 "'": 34,
 '-': 24,
 '.': 33,
 '/': 49,
 '0': 38,
 '1': 35,
 '2': 42,
 '3': 41,
 '4': 43,
 '5': 37,
 '6': 45,
 '7': 46,
 '8': 44,
 '9': 36,
 '<': 28,
 '</s>': 2,
 '<space>': 1,
 '>': 29,
 'N': 30,
 '\\': 48,
 'a': 3,
 'b': 6,
 'c': 14,
 'd': 23,
 'e': 4,
 'f': 19,
 'g': 21,
 'h': 22,
 'i': 12,
 'j': 31,
 'k': 8,
 'l': 11,
 'm': 20,
 'n': 7,
 'o': 9,
 'p': 26,
 'q': 25,
 'r': 5,
 's': 18,
 't': 10,
 'u': 17,
 'v': 32,
 'w': 15,
 'x': 27,
 'y': 16,
 'z': 13}